<a href="https://colab.research.google.com/github/NicoMedinaa/LaboratorioVentas/blob/main/Laboratorio_de_An%C3%A1lisis_de_ventas_mensuales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importo Librerias


In [ ]:
import pandas as pd
import os
import plotly.graph_objects as go

# Cargo el dataframe y lo limpio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
rutaDeAcceso = "/content/drive/MyDrive/Colab Notebooks/dataset-ventas"

archivos = os.listdir(rutaDeAcceso) #Creo un dataframe vacio para poder unir todos los dataframes

dataframes = {}
for archivo in archivos: #Recorro todos los archivos que se encuentran en la carpeta
    if archivo.endswith('.csv'):  # Asegurarse de que solo se consideren archivos CSV
        ruta_completa = os.path.join(rutaDeAcceso, archivo)  # Ruta completa de cada archivo individual
        mes = archivo.split("_")[3].split(".")[0]
        df = pd.read_csv(ruta_completa) #Guarda el dataframe del mes

        df['Cantidad Pedida'] = pd.to_numeric(df['Cantidad Pedida'], errors='coerce')  #El errors= 'coerce' transforma los valores no numericos a NaN'
        df['Precio Unitario'] = pd.to_numeric(df['Precio Unitario'], errors='coerce')

        df = df.loc[~df['Cantidad Pedida'].isna()]
        df = df.loc[~df['Precio Unitario'].isna()]

        # Elimino todos los Nan y me quedo solo con los valores numericos
        df = df.dropna()

        # Transformo todos los valores numericos al tipo de dato int
        df['Cantidad Pedida'] = df['Cantidad Pedida'].astype(int)
        df['Precio Unitario'] = df['Precio Unitario'].astype(int)

        dataframes[mes] = df #Y loego lo concateno con el resto de los dataframes
#dataframes #Como resultado tengo un diccionario con cada mes como key y el value es el dataframe de ventas de ese mes

# Creo un dataframe general

In [ ]:
meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
#Creo una lista de messes
dfOrdenado = pd.DataFrame() #Creo un dataframe vacio

for mes in meses: #Recorro por cada mes
  dfOrdenado = pd.concat([dfOrdenado, dataframes[mes]], axis=0) #Y concateno cada dataframe

dfOrdenado = dfOrdenado.reset_index() #Reseteo los indices

# Metodo para la extracción de datos

In [11]:
def extraer_datos(datos):
  datos["Fecha de Pedido"] = pd.to_datetime(datos["Fecha de Pedido"])
  datos["Hora"] = datos["Fecha de Pedido"].dt.hour
  datos["Dia"] = datos["Fecha de Pedido"].dt.day
  return datos

#### 1. Comportamiento de las ventas en los distintos meses:
● ¿Cómo variaron las ventas a lo largo de los diferentes meses?
● ¿Hubo algún mes que se destacó en términos de ventas?
● ¿Cuál es el ingreso total generado por mes?

In [ ]:
# Creo listas para almacenar los totales de ventas y los nombres de los meses
totales_ventas = []
nombres_meses = []
total_ingreso = []

for mes in meses:
    dataframe_meses = dataframes[mes]
    dataframe_meses = extraer_datos(dataframe_meses)

    total_ventas_mes = dataframe_meses['Cantidad Pedida'].sum()
    totales_ventas.append(total_ventas_mes)
    nombres_meses.append(mes)

    total_ingreso_mes = dataframe_meses['Precio Unitario'].sum()
    total_ingreso.append(total_ingreso_mes)

# Creo el objeto figura con el gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x=nombres_meses,
    y=totales_ventas,
    marker=dict(color=totales_ventas, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=totales_ventas,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas por mes",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Total de ventas"),
)


fig.show()

fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=nombres_meses,
    y=total_ingreso,
    marker=dict(color=total_ingreso, colorscale = 'RdYlGn', colorbar=dict(title="Ingreso de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=total_ingreso,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig2.update_layout(
    title="Ingresos por mes",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Ingresos de ventas"),
)

#### 2. Optimización de la publicidad y patrón de ventas por hora:
● ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la
probabilidad de compra?
● ¿Cómo cambian los patrones de ventas por hora a lo largo del año?
● ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad
en los distintos meses?

In [ ]:
Horas = list(range(24))

# Creo el objeto figura
fig = go.Figure()

for mes in meses:
    dataframe_mes_hora = dataframes[mes]
    dataframe_mes_hora = extraer_datos(dataframe_mes_hora)  # Reemplaza esto con el procesamiento que necesites
    Ventas_por_hora = dataframe_mes_hora.groupby('Hora')['Cantidad Pedida'].count()



    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=Horas,
        y=Ventas_por_hora,
        mode='lines',
        name=mes,
        line=dict(width=2,shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora en diferentes meses",
    xaxis=dict(title="Hora", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

Horas = list(range(24))

# Creo el objeto figura
fig = go.Figure()
horas_analizadas = [19, 12, 16]

for hora in horas_analizadas:
    total_productos_vendidos = []

    for mes in meses:
        dataframe_mes_hora = dataframes[mes]
        dataframe_mes_hora = extraer_datos(dataframe_mes_hora)  # Reemplaza esto con el procesamiento que necesites

        ventas_hora = dataframe_mes_hora[dataframe_mes_hora['Hora'] == hora]

        # Calcular la cantidad total de productos vendidos para la hora actual
        total_productos_vendidos.append(ventas_hora['Cantidad Pedida'].count())

    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=meses,
        y=total_productos_vendidos,
        mode='lines',
        name=f'{hora} horas',
        line=dict(width=2, shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora",
    xaxis=dict(title="Meses", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()

# 3. Ventas por ciudades

In [ ]:
ciudades = [] #Creo una lista de ciudades
for id in range(0 ,len(dfOrdenado)): #Recorro todo el dataframe
  ciudades.append(dfOrdenado['Dirección de Envio'][id].split(', ')[1]) #Guardo en la lista el nombre de la ciudad unicamente


dfOrdenado['Ciudades'] = ciudades #Creo una nueva lista con la ciudad donde se realizo cada compra

In [ ]:
ventasCiudades = dfOrdenado.groupby("Ciudades")["Cantidad Pedida"].count() #Agrupo a cada ciudad la cantidad de ventas que se realizaron


fig = go.Figure()

fig.add_trace(go.Bar(
    x=ventasCiudades.keys(), #Como X uso las keys de las ciudades
    y=ventasCiudades, #Y como Y uso las ventas de cada una originadas por el groupby
    marker=dict(color=ventasCiudades, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=ventasCiudades,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas por ciudades",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Total de ventas"),
)

fig.show()

Georgia = Atlanta
Texas = Austin, Dallas
Massachusetts = Boston
California = Los Angeles, San Francisco
New york = new york city
Oregón = Portland
Washington = seattle

In [ ]:
estados = pd.Series(dtype='float64') #Creo una serie con los estados y los agrupo para asi facilitar la muestra
estados["Georgia"] = ventasCiudades["Atlanta"]
estados["Texas"] = ventasCiudades["Austin"] + ventasCiudades["Dallas"]
estados["Massachusetts"] = ventasCiudades["Boston"]
estados["California"] = ventasCiudades["Los Angeles"] + ventasCiudades["San Francisco"]
estados["New York"] = ventasCiudades["New York City"]
estados["Oregón"] = ventasCiudades["Portland"]
estados["Washington"] = ventasCiudades["Seattle"]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=estados.keys(),
    y=estados,
    marker=dict(color=estados, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=estados,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas por estado",
    xaxis=dict(title="Estado"),
    yaxis=dict(title="Total de ventas"),
)

fig.show()

In [ ]:
mesdF = [] #Creo una lista de meses
for id in range(0 ,len(dfOrdenado)):
  mesdF.append(dfOrdenado['Fecha de Pedido'][id].split('/')[0]) #Guardo unicamente el numero de cada mes


dfOrdenado['Meses'] = mesdF #Creo una nueva columna identificatoria de cada mes

In [ ]:
df_grouped = dfOrdenado.groupby(['Meses', 'Ciudades'])['Cantidad Pedida'].sum().unstack() #Hago un grupby para sacar la cantidad de ventas en cada ciudad por mes

### 4. Análisis del producto más vendido:
● ¿Cuál es el producto más vendido en general y en cada mes?
● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?

In [ ]:
productos = pd.DataFrame()
producto_mes = []
producto_mes_cantidad=[]
producto_mes_articulo=[]
for mes in meses:
  productos =dataframes[mes].groupby('Producto')['Cantidad Pedida'].sum()
  producto_mes.append(productos.idxmax())
  producto_mes_cantidad.append(productos.max())


fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=nombres_meses,
    y=producto_mes_cantidad,
    marker=dict(color=producto_mes_cantidad, colorscale = 'RdYlGn', colorbar=dict(title="Ingreso de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=producto_mes,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig2.update_layout(
    title="Producto mas Vendido por mes",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Ventas del Producto"),
)

fig2.show()

In [ ]:
fig = go.Figure()


for estado in df_grouped.columns:
    fig.add_trace(go.Bar(x=meses, y=df_grouped[estado], name=estado)) #Al hacer el grafico uso como X a los meses


fig.show()

# 5-Tendencia Ventas

In [15]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

VentasDia={}
fig = go.Figure()
for mes in meses:
    dataframeMes = dataframes[mes]
    dataframeMes = extraer_datos(dataframeMes)  # Reemplaza esto con el procesamiento que necesites
    VentasDia[mes] = dataframeMes.groupby("Dia")["Cantidad Pedida"].sum()




    # Agrego al objeto figura un grafico de de linea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(

        x=list(range(1,31,1)),
        y=VentasDia[mes],
        mode='lines',
        name=mes,

        line=dict(shape='spline', smoothing=1.3, width=2)))


  # Configuro los ejes y los títulos
fig.update_layout(

    title="Unidades vendidas por Dia",
    xaxis=dict(title="Dias", tickvals=list(range(1,31,1))),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis=dict(title="Unidades vendidas", side="left"))


fig.show()

# 6-Dias festivos

In [ ]:
VentasDia = {} #Creo un diccionario

for mes in meses: #Recorro cada uno de los meses para obtener la cantidad de pedidos en cada uno de los dias de cada mes
    dataframeMes = dataframes[mes]
    dataframeMes = extraer_datos(dataframeMes)
    VentasDia[mes] = dataframeMes.groupby("Dia")["Cantidad Pedida"].count()

In [ ]:
fig = go.Figure()

diasLista = ["1 de Enero", "14 de Febrero","4 de Julio","17 de Marzo", "24 de Diciembre","25 de Diciembre"] #Lista con los dias a utilizarse en el eje X del grafico

valores = pd.Series([VentasDia["Enero"].iloc[0], VentasDia["Febrero"].iloc[13], VentasDia["Marzo"].iloc[16], VentasDia["Julio"].iloc[3], VentasDia["Diciembre"].iloc[23] ,VentasDia["Diciembre"].iloc[24]] )
#Creo una serie con los valores importantes para el analisis
fig.add_trace(go.Bar(
    x=diasLista,
    y=valores,
    marker=dict(color=valores, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=valores,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas por dia Festivo",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Dia"),
)

fig.show()